In [1]:
import functions.wiwb as wiwb
import pandas as pd
import plotly.express as px
import geopandas as gpd

from datetime import datetime, timedelta
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
import os
os.environ["WIWB_USERNAME"] = "wsbd"
os.environ["WIWB_PASSWORD"] = "4B83lSttDBity1kBtYzO"

In [8]:
start = datetime(2021,4,1)

locs = gpd.read_file("../01_data/ground_stations.gpkg")
locs = locs[locs['use']==True]
points, extent = wiwb.get_points_from_gdf(locs)
wiwb_realtime = pd.Series(dtype="float64")
wiwb_early = pd.Series(dtype="float64")

while start < datetime(2022,5,1):
    end = start + timedelta(days=10)
    print(f"downloading from {start} to {end}", end="\r")

    df_realtime = wiwb.download_wiwb(data_source="KNMI IRC Realtime", 
                                    points=points, 
                                    start=start, 
                                    end=end, 
                                    extent=extent)
    if wiwb_realtime.empty:
        wiwb_realtime = df_realtime.copy()
    else:
        wiwb_realtime = wiwb_realtime.append(df_realtime)

    df_early = wiwb.download_wiwb("KNMI IRC Early Reanalysis", points, start, end, extent)
    if wiwb_early.empty:
        wiwb_early = df_early.copy()
    else:
        wiwb_early = wiwb_early.append(df_early)

    start = end

In [7]:
pd.DataFrame(wiwb_realtime).to_parquet("../01_data/wiwb/irc_realtime_raw.parquet")
pd.DataFrame(wiwb_early).to_parquet("../01_data/wiwb/irc_early_raw.parquet")

In [13]:
px.line(wiwb_realtime.unstack(level=1).resample("d").sum().cumsum())

In [14]:
px.line(wiwb_early.unstack(level=1).resample("d").sum().cumsum())